# cuGraph Intro

*Original Authors: Bradley Rees and James Wyles. Updated by Adam Breindel*


## Single Source Shortest Path (SSSP)

First, we will use cuGraph to compute the shortest path from a starting vertex to everyother vertex in our training dataset.

Single source shortest path computes the shortest paths from the given starting vertex to all other reachable vertices. 

To compute SSSP for a graph in cuGraph we use:
**cugraph.sssp(G, source)**

Input
* __G__: cugraph.Graph object
* __source__: int, Index of the source vertex

Returns 
* __df__:  a cudf.DataFrame object with two columns:
    * df['vertex']: The vertex identifier for the vertex
    * df['distance']: The computed distance from the source vertex to this vertex
    



### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](https://materials.s3.amazonaws.com/i/zgraph.png)

This is a small graph which allows for easy visual inspection to validate results.  
__Note__: The Karate dataset starts with vertex ID 1 which the cuGraph analytics assume a zero-based starting ID.  

In [74]:
import cugraph
import cudf
from collections import OrderedDict

### Read the data and adjust the vertex IDs

In [75]:
! head data/karate-data.csv

1	2
1	3
1	4
1	5
1	6
1	7
1	8
1	9
1	11
1	12


In [76]:
import os

datafile=os.getcwd() + '/data/karate-data.csv'

In [77]:
import cudf

karate = cudf.read_csv(datafile, names=['src','dst'], delimiter='\t', dtype={'src':'int32', 'dst':'int32'} )

In [78]:
karate.head().to_pandas()

,src,dst
0,1,2
1,1,3
2,1,4
3,1,5
4,1,6


In [79]:
import cugraph

G = cugraph.Graph()
G.add_edge_list(karate["src"], karate["dst"])

## Breadth First Search (BFS) 

Now, we'll generalize and use cuGraph to compute the Breadth First Search path from a starting vertex to everyother vertex in our training dataset.

As the name implies, BFS traverses the given graph in a breadth first manner. Starting at a specified vertex, the algorithms iteratively searches neighboring vertices.  


@see https://en.wikipedia.org/wiki/Breadth-first_search


To compute BFS in cuGraph use: __bfs(G, start_id)__

* __G__: A cugraph.Graph object
* __start_id__ : the starting vertex ID

Returns:

* __df__: cudf.DataFrame with three names columns:
    * df["vertex"]:   The vertex id.
    * df["distance"]: The distance to the starting vertex
    * df["predecessor"]: The vertex ID of the vertex that was used to reach this vertex

In [80]:
# Call BFS on the graph starting from vertex 1
df = cugraph.bfs(G,1)
df

,vertex,distance,predecessor
0,0,2147483647,-1
1,1,0,-1
2,2,1,1
3,3,1,1
4,4,1,1
5,5,1,1
6,6,1,1
7,7,1,1
8,8,1,1
9,9,1,1


In [81]:
# define a print path function that take the dataframe and a vertex ID

def print_path(df, id):
    
    # Use the BFS predecessors and distance to trace the path 
    # from vertex id back to the starting vertex ( vertex 1 in this example)
    dist = df['distance'][id]
    lastVert = id
    for i in range(dist):
        nextVert = df['predecessor'][lastVert]
        d = df['distance'][lastVert]
        print("Vertex: " + str(lastVert) + " was reached from vertex " + str(nextVert) + 
        " and distance to start is " + str(d) )
        lastVert = nextVert

In [82]:
print_path(df, 22)

Vertex: 22 was reached from vertex 1 and distance to start is 1


In [83]:
print_path(df, 30)

Vertex: 30 was reached from vertex 33 and distance to start is 3
Vertex: 33 was reached from vertex 9 and distance to start is 2
Vertex: 9 was reached from vertex 1 and distance to start is 1


### Since we can see in graph illustraion above that vertex 17 is at the edge of the graph, let's run BFS with that as the startring vertex

In [84]:
# Call BFS on the graph starting from vertex 17
df2 = cugraph.bfs(G,17)

In [85]:
# Print the max distance
df2["distance"].max()

2147483647

Notice that max returned an unexpected value.  That is becouse the isolated vertex, 0, is unreachable.

> Whenever a graph contains disjoint components, the distance to the unconnected vertices will always be max_int

In [86]:
df2["distance"][0]

2147483647

In [87]:
# drop all large distances 

df3 = df2.query("distance < 100")

In [88]:
# Print the max distance
df3["distance"].max()

5

In [89]:
# Print path to vertex 30

print_path(df2, 30)

Vertex: 30 was reached from vertex 33 and distance to start is 5
Vertex: 33 was reached from vertex 3 and distance to start is 4
Vertex: 3 was reached from vertex 1 and distance to start is 3
Vertex: 1 was reached from vertex 6 and distance to start is 2
Vertex: 6 was reached from vertex 17 and distance to start is 1


### Create a Graph and call SSSP

In [90]:
# Call cugraph.sssp to get the distances from vertex 0:
df4 = cugraph.sssp(G, 1)
df4

,vertex,distance,predecessor
0,0,3.402823e+38,-1
1,1,0.000000e+00,-1
2,2,1.000000e+00,1
3,3,1.000000e+00,1
4,4,1.000000e+00,1
5,5,1.000000e+00,1
6,6,1.000000e+00,1
7,7,1.000000e+00,1
8,8,1.000000e+00,1
9,9,1.000000e+00,1


# Find the farthest vertex from the source using the distances:

In [91]:
df4.sort_values('distance', ascending=False)

,vertex,distance,predecessor
0,0,3.402823e+38,-1
15,15,3.000000e+00,34
16,16,3.000000e+00,34
19,19,3.000000e+00,34
21,21,3.000000e+00,34
23,23,3.000000e+00,34
24,24,3.000000e+00,34
27,27,3.000000e+00,34
30,30,3.000000e+00,34
10,10,2.000000e+00,3


## Shortest Paths with (Asymmetric) Costs

BFS looks a lot liks Shortest Paths when all of the edges have weight 1.0

Let's see how this looks if we make edge costs from (but not to) node 1 be 1.5


In [92]:
karate.head()

,src,dst
0,1,2
1,1,3
2,1,4
3,1,5
4,1,6


In [93]:
karate['weight'] = 1.0

In [94]:
karate.head()

,src,dst,weight
0,1,2,1.0
1,1,3,1.0
2,1,4,1.0
3,1,5,1.0
4,1,6,1.0


In [95]:
karate.loc[karate.src == 3, 'weight'] = 0.5

In [96]:
karate

,src,dst,weight
0,1,2,1.0
1,1,3,1.0
2,1,4,1.0
3,1,5,1.0
4,1,6,1.0
5,1,7,1.0
6,1,8,1.0
7,1,9,1.0
8,1,11,1.0
9,1,12,1.0


In [97]:
G2 = cugraph.Graph()
G2.add_edge_list(karate.src, karate.dst, value_col=karate.weight)

In [98]:
df5 = cugraph.sssp(G2, 1)
df5

,vertex,distance,predecessor
0,0,1.797693e+308,-1
1,1,0.000000e+00,-1
2,2,1.000000e+00,1
3,3,1.000000e+00,1
4,4,1.000000e+00,1
5,5,1.000000e+00,1
6,6,1.000000e+00,1
7,7,1.000000e+00,1
8,8,1.000000e+00,1
9,9,1.000000e+00,1


For comparison:

In [99]:
df4.sort_values('distance', ascending=False).head(10)

,vertex,distance,predecessor
0,0,3.402823e+38,-1
15,15,3.000000e+00,34
16,16,3.000000e+00,34
19,19,3.000000e+00,34
21,21,3.000000e+00,34
23,23,3.000000e+00,34
24,24,3.000000e+00,34
27,27,3.000000e+00,34
30,30,3.000000e+00,34
10,10,2.000000e+00,3


In [100]:
df5.sort_values('distance', ascending=False).head(10)

,vertex,distance,predecessor
0,0,1.797693e+308,-1
27,27,3.000000e+00,34
15,15,2.500000e+00,33
16,16,2.500000e+00,33
19,19,2.500000e+00,33
21,21,2.500000e+00,33
23,23,2.500000e+00,33
24,24,2.500000e+00,28
30,30,2.500000e+00,33
17,17,2.000000e+00,6
